In [1]:
! pip install --quiet langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


# Structured Output Parser.

This output parser can be used when you want to return multiple fields.

In [14]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

**Response schema tells LLM how i want my output to be formatted.**

Example:
```
{
  'answer': 'The capital of India is New Delhi.',
 'source': 'https://en.wikipedia.org/wiki/New_Delhi'
}
```

In [5]:
respose_schemas = [
    ResponseSchema(name = 'answer',
                   description='answer to the users question.'),
    ResponseSchema(
        name = "source",
        description='source used to answer users question, should be a website link.'
    )
    ]

Add the schema structure to the output parser.

In [6]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas=respose_schemas)

We now get a string that contains instructions for how the response should be formatted, and we then insert that into our prompt.

In [8]:
format_instructions = output_parser.get_format_instructions()

In [9]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the users question.\n\t"source": string  // source used to answer users question, should be a website link.\n}\n```'

Pass the formatted instructions to prompt

In [10]:
template='''
  answer the users question as best as possible.
  {format_instructions}
  {question}
'''

In [12]:
prompt = PromptTemplate(
    template=template,
    input_variables=['question'],
    partial_variables={'format_instructions' : format_instructions}
)

In [13]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
model = ChatOpenAI(temperature=0)

In [15]:
chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=output_parser
)

In [16]:
response = chain.invoke({"question": "what's the capital of India?"})

In [23]:
response

{'question': "what's the capital of India?",
 'text': {'answer': 'The capital of India is New Delhi.',
  'source': 'https://en.wikipedia.org/wiki/New_Delhi'}}

In [22]:
response['text']

{'answer': 'The capital of India is New Delhi.',
 'source': 'https://en.wikipedia.org/wiki/New_Delhi'}

In [24]:
response = chain.invoke({"question": "what's the capital of Nepal?"})

In [25]:
response['text']

{'answer': 'The capital of Nepal is Kathmandu.',
 'source': 'https://en.wikipedia.org/wiki/Kathmandu'}